In [2]:
import numpy as np
import math
import re
import itertools
import pandas as pd
import time
import os
import pickle
import copy
from Functions import *

In [3]:
N=6
dct_cmbn={}
dct_cmbn.update({1:np.arange(N)})
for j in range(2,5):
    dct_cmbn.update({j:np.array(list(itertools.combinations(np.arange(N),j)))})

In [22]:
def add_1(combinantions_dictionary, ND=5):
    N=combinantions_dictionary[1][-1]+1
    new_cd={1:np.append(combinantions_dictionary[1],N)}
    diff=1
    while diff<(ND):
        new_part=np.vstack([combinantions_dictionary[diff].T,np.array([N]*len(combinantions_dictionary[diff]))])
        new_in=np.hstack([combinantions_dictionary[diff+1].T,new_part]).T
        new_cd.update({(diff+1):new_in})

        diff+=1
    

    return(new_cd)


In [23]:
N=34
dct_cmbn={}
dct_cmbn.update({1:np.arange(N)})
for j in range(2,6):
    dct_cmbn.update({j:np.array(list(itertools.combinations(np.arange(N),j)))})

In [24]:
F_dict={6:dct_cmbn}

In [25]:
def add_N(full_dict,N_start, N_add):
    tmp_d=full_dict[N_start]
    i=0
    while i<N_add:
        tmp_d=add_1(tmp_d)
        full_dict.update({(N_start+i+1):tmp_d})
        i+=1

    return(full_dict)


In [26]:
def iterative_add_N(dict_start, N_add, save=True,save_dir='./combinations/'):
    tmp_d=copy.deepcopy(dict_start)
    N_start=dict_start[1][-1]
    i=0
    while i<N_add:
        print(i)
        tmp_d=add_1(tmp_d)
        if save:
            NM=save_dir+'N_'+str(N_start+i+1)+'.pk'
            with open(NM, 'wb') as handle:
                pickle.dump(tmp_d, handle, protocol=pickle.HIGHEST_PROTOCOL)
        i+=1

    return(tmp_d)

In [27]:
WA_mat=assign_wells_mat(n_compounds=35)


In [28]:
WA_mat.shape

(35, 12)

In [29]:
dct_cmbn[4].shape

(46376, 4)

In [30]:
N=35
diff=6

In [31]:
dct_cmbn={}
dct_cmbn.update({1:np.arange(N)})
for j in range(2,diff+1):
    dct_cmbn.update({j:np.array(list(itertools.combinations(np.arange(N),j)))})

In [32]:
f1=add_1(dct_cmbn, ND=diff)

In [33]:
f1[4].shape

(58905, 4)

In [34]:
list(dct_cmbn)

[1, 2, 3, 4, 5, 6]

In [35]:
list(f1)

[1, 2, 3, 4, 5, 6]

In [13]:
WA_mat[f1[4]].shape

(52360, 4, 12)

In [14]:
sm=np.bool(np.sum(WA_mat[f1[4]], axis=1))

In [15]:
sm[0,:]

array([ True, False, False, False, False, False,  True,  True,  True,
        True, False, False])

In [16]:
sm.shape

(52360, 12)

In [17]:

#NMO=save_dir+'N_'+str(N)+'.pk'
NMO='./combinations/N_294.pk'
with open(NMO, "rb") as input_file:
    scrambler = pickle.load(input_file)

In [18]:
scrambler

{1: array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 

In [ ]:
def is_consistent_precomp(well_assigner:np.array, differentiate:int, scrambler:dict) -> list:
    if differentiate==0:
        return(True,well_assigner, np.array([1]*well_assigner.shape[0]))
    N=well_assigner.shape[0]
    for i in range(differentiate):
        diff=i+1
        if diff ==1:
            full_well_assigner=well_assigner.copy()
        else:
            this_sc=scrambler[diff]
            full_well_assigner=np.concatenate((full_well_assigner,np.bool(np.sum(well_assigner[this_sc], axis=1))))
    _, counts=np.unique(full_well_assigner, axis=0, return_counts=True)
    if len(counts)<full_well_assigner.shape[0]:
        return(False, full_well_assigner, counts)
    elif len(counts)==full_well_assigner.shape[0]:
        return(True,full_well_assigner, counts)
    else:
        print("Something is fishy")
        return(-1)
    
#
def mean_metrics_precomp(well_assigner, differentiate, scrambler, **kwargs):
    BT=well_assigner.shape[1]
    _,_, counts= is_consistent_precomp(well_assigner, differentiate, scrambler) 
    ET=extra_tests(counts)   
    rounds=np.sum(counts>1)/np.sum(counts>0)+1
    p_check=np.round(np.sum(counts[counts>1])/np.sum(counts)*100)
    return BT+ET, ET,  rounds, p_check

def decode_precomp(well_assigner:np.ndarray, readout:np.ndarray, differentiate:int, scrambler:dict) -> list:
    N=well_assigner.shape[0]
    for i in range(differentiate):
        resulti=[]
        diff=i+1
        if diff ==1:
            resulti.extend(list(range(well_assigner.shape[0])))
            full_well_assigner=well_assigner.copy()
        else:
            this_sc=scrambler[diff]
            full_well_assigner=np.concatenate((full_well_assigner,np.bool(np.sum(well_assigner[this_sc], axis=1))))
    idxs=[i for i in range(full_well_assigner.shape[0]) if np.array_equal(full_well_assigner[i,:],readout)]
    if len(idxs)==0:
        print('No match')
        return(-1)
    
    return [resulti[i] for i in idxs]

def find_rand_params_precomp(n_compounds:int, differentiate:int, scrambler:dict, n_compounds_per_well=0, n_wells=0, guesses=0, 
                     max_compounds=0, max_redundancy=4, min_redundancy=1):
    skip_compounds=True
    skip_wells=True
    if n_compounds_per_well==0:
        skip_compounds=False
    if n_wells==0:
        skip_wells=False
    if guesses==0:
        guesses=n_compounds

    MC= int(n_compounds/2) if max_compounds==0 else max_compounds
    mc=int(np.sqrt(n_compounds)) if int(np.sqrt(n_compounds))<MC else int(MC/2)
    arr_comp=np.arange(int(mc),int(MC+1))
    mw=int(np.log2(n_compounds))
    MW=int(2*np.sqrt(n_compounds))
    while MW-mw<10:
        mw=int(abs(mw-1))
        MW=int(MW+1)

    arr_wells=np.arange(mw,MW)
    min_tests=np.inf
    for comp in arr_comp:
        if skip_compounds:
            comp=n_compounds_per_well

        for wells in arr_wells:
            if skip_wells:
                if skip_compounds:
                    
                    return n_compounds_per_well, n_wells, assign_wells_random_precomp(n_compounds=n_compounds, differentiate=differentiate, 
                                               n_compounds_per_well=n_compounds_per_well, n_wells=n_wells, guesses=guesses, Evaluate=True, 
                                               scrambler=scrambler)
                wells=n_wells
                
            if comp*wells>max_redundancy*n_compounds*np.log2(n_compounds) or comp*wells<min_redundancy*n_compounds: continue 
            WA_tmp, mean_exp=assign_wells_random_precomp(n_compounds, differentiate, comp, wells,
                                                          guesses, Evaluate=True, return_me=True,scrambler=scrambler)
            if mean_exp<min_tests:
                Comp=comp
                Wells=wells
                min_tests=mean_exp
                min_wa=WA_tmp
            if skip_wells:
                break
        if skip_compounds:
            break

    return Comp, Wells, min_tests, min_wa

def assign_wells_random_precomp(n_compounds:int,  differentiate:int,scrambler:dict, n_compounds_per_well=0, 
                        n_wells=0, guesses=0, Evaluate=False, return_me=False, **kwargs)->np.array:
    if guesses==0:
        guesses=n_compounds
    min_tests=np.inf

    if n_compounds_per_well==0 or n_wells==0:
        _,_, min_tests, WA_rand=find_rand_params_precomp(n_compounds=n_compounds, differentiate=differentiate, 
                                 n_compounds_per_well=n_compounds_per_well, n_wells=n_wells, guesses=guesses,
                                 scrambler=scrambler)
        if return_me:
            return WA_rand,  min_tests
        
        return WA_rand
        


    if Evaluate:
        second_axis=np.tile(np.arange(n_wells),n_compounds_per_well).reshape(n_compounds_per_well,-1)
        for i in range(guesses):
            idt=np.random.randint(0,n_compounds,size=(n_compounds_per_well,n_wells) )
            well_assigner=np.zeros((n_compounds,n_wells))==1
            well_assigner[idt, second_axis]=True
            if guesses==1:
                if return_me:
                    mean_exp, _, _, p_check= mean_metrics_precomp(well_assigner=well_assigner, 
                                                                differentiate=differentiate,scrambler=scrambler)
                    return well_assigner, mean_exp
                return well_assigner
            mean_exp, _, _, p_check= mean_metrics_precomp(well_assigner=well_assigner,
                                                        differentiate=differentiate, scrambler=scrambler)
            if p_check<1:
                if return_me:
                    return well_assigner,  mean_exp
                return well_assigner
            elif mean_exp<min_tests: 
                best_wa=well_assigner.copy()
                min_tests=mean_exp

        if return_me:
            return best_wa,  min_tests
        
        return best_wa

    _,_, min_tests, WA_rand=find_rand_params_precomp(n_compounds=n_compounds, differentiate=differentiate, 
                                 n_compounds_per_well=n_compounds_per_well, n_wells=n_wells, 
                                 guesses=guesses,scrambler=scrambler)
    if return_me:
        return WA_rand,  min_tests
    
    return WA_rand


In [ ]:
def full_sweep_deterministic_WAs(start=50, stop=150, step=10, **kwargs):
    dict_comp={}
    current=start
    kwargs['return_wa']=True
    while current<stop:
        time0=time.time()
        if kwargs['timeit']:
            print(current)
        df_met, dict_wa=full_deterministic_WAS(n_compounds=current, **kwargs)
        dict_comp.update({current:[df_met, dict_wa]})
        current=current+step
        if kwargs['timeit']:
            print("segment time: %s seconds" % np.round(time.time() - time0, 1))
    return dict_comp


def full_deterministic_WAS(**kwargs):
    methods=['matrix', 'random', 'STD', 'Chinese trick']
    # matrix assignment
    

    # multidimensional matrix
    if 'n_dims' in kwargs.keys():
        WA_mul=assign_wells_multidim(**kwargs)
        ndmin=kwargs['n_dims']
        multi=['multidim: '+str(ndmin)]
        WA_list=[WA_mul]
    elif 'all_dims' in kwargs.keys():
        if kwargs['all_dims']:
            WA_list=[]
            multi=[]
            for i in np.arange(2,int(np.ceil(np.log(kwargs['n_compounds'])/np.log(2)))):
                if i>kwargs['max_dims']:
                    continue
                WA_mul=assign_wells_multidim(n_dims=i, **kwargs)
                WA_list.append(WA_mul)
                multi.append('multidim_'+str(i))

        else:
            ndmin= find_dims(**kwargs)
            WA_mul=assign_wells_multidim(n_dims=ndmin, **kwargs)
            multi=['multidim_'+str(ndmin)]
            WA_list=[WA_mul]
            


    else:
        ndmin= find_dims(**kwargs)
        WA_mul=assign_wells_multidim(n_dims=ndmin, **kwargs)
        multi=['multidim: '+str(ndmin)]
        WA_list=[WA_mul]
    
    multi.extend(methods)
    methods=multi.copy()

    WA_mat=assign_wells_mat(**kwargs)

    # random assignment

    WA_ran=assign_wells_random(**kwargs)

    # STD asignment 
    WA_std=assign_wells_STD(**kwargs)

    

    # chinese trick assignment
    WA_chin=assign_wells_chinese(**kwargs)


    WA_list.extend([WA_mat, WA_ran,WA_std, WA_chin])

    WA_bin=assign_wells_L(**kwargs)
    methods.append('Binary')
    WA_list.append(WA_bin)
    this_dir=os.path.join(kwargs['save_dir'],'N_'+str(kwargs['n_compounds']))
    if not os.path.exists(this_dir):
        os.makedirs(this_dir)

    for method, WA in zip(methods, WA_list)



    #hierarchical


In [ ]:
def undiff_WAS(**kwargs):
    methods=['matrix', 'random', 'STD', 'Chinese trick']
    # matrix assignment
    

    # multidimensional matrix
    if 'n_dims' in kwargs.keys():
        WA_mul=assign_wells_multidim(**kwargs)
        ndmin=kwargs['n_dims']
        multi=['multidim: '+str(ndmin)]
        WA_list=[WA_mul]
    elif 'all_dims' in kwargs.keys():
        if kwargs['all_dims']:
            WA_list=[]
            multi=[]
            for i in np.arange(2,int(np.ceil(np.log(kwargs['n_compounds'])/np.log(2)))):
                if i>kwargs['max_dims']:
                    continue
                WA_mul=assign_wells_multidim(n_dims=i, **kwargs)
                WA_list.append(WA_mul)
                multi.append('Multidim_'+str(i))

        else:
            ndmin= find_dims(**kwargs)
            WA_mul=assign_wells_multidim(n_dims=ndmin, **kwargs)
            multi=['Multidim_'+str(ndmin)]
            WA_list=[WA_mul]
            


    else:
        ndmin= find_dims(**kwargs)
        WA_mul=assign_wells_multidim(n_dims=ndmin, **kwargs)
        multi=['multidim: '+str(ndmin)]
        WA_list=[WA_mul]
    
    multi.append('Matrix')
    methods=multi.copy()

    WA_mat=assign_wells_mat(**kwargs)


    WA_list.append(WA_mat)

    if kwargs['differentiate']<2:
        WA_bin=assign_wells_L(**kwargs)
        methods.append('Binary')
        WA_list.append(WA_bin)

    #hierarchical
        
    Hier=calculate_metrics_hierarchical(**kwargs)
    #return([BM[0], BM[1],layers, MC, details ])

    ls_met=[]
    ls_names_met=['mean_experiments', 'max_compounds_per_well', 'n_wells', 'percentage_check', 'mean_extra_exp', 'mean_steps']
    for method, WA in zip(methods, WA_list):
        mean_exp, extra_exp,  _, perc_check= mean_metrics(WA, **kwargs)
        n_wells=WA.shape[1]
        M_exp=np.round(mean_exp, 2)
        max_comp=np.max(np.sum(WA, axis=0))
        ls_met.append([M_exp, max_comp, n_wells, int(perc_check),  extra_exp,1+perc_check/100])
    ls_met.append(Hier[:-1])
    full_methods=methods.copy()
    full_methods.append('Hierarchical')
    df_met=pd.DataFrame(ls_met)

    dict_wa={method: WA for method, WA in zip(methods, WA_list)}
    dict_wa.update({'Hierarchical':Hier[5]})

    idx_renamer={i:j for i,j in zip(df_met.index, full_methods)}
    col_renamer={i:j for i,j in zip(df_met.columns, ls_names_met)}
    df_met.rename(index=idx_renamer, columns=col_renamer, inplace=True)

    ret_wa= kwargs['return_wa'] 
    if ret_wa:
        return df_met, dict_wa
    return df_met

In [ ]:
def full_deterministic_WAS(**kwargs):
    methods=['matrix', 'random', 'STD', 'Chinese trick']
    # matrix assignment
    

    # multidimensional matrix
    if 'n_dims' in kwargs.keys():
        WA_mul=assign_wells_multidim(**kwargs)
        ndmin=kwargs['n_dims']
        multi=['multidim: '+str(ndmin)]
        WA_list=[WA_mul]
    elif 'all_dims' in kwargs.keys():
        if kwargs['all_dims']:
            WA_list=[]
            multi=[]
            for i in np.arange(2,int(np.ceil(np.log(kwargs['n_compounds'])/np.log(2)))):
                if i>kwargs['max_dims']:
                    continue
                WA_mul=assign_wells_multidim(n_dims=i, **kwargs)
                WA_list.append(WA_mul)
                multi.append('multidim: '+str(i))

        else:
            ndmin= find_dims(**kwargs)
            WA_mul=assign_wells_multidim(n_dims=ndmin, **kwargs)
            multi=['multidim: '+str(ndmin)]
            WA_list=[WA_mul]
            


    else:
        ndmin= find_dims(**kwargs)
        WA_mul=assign_wells_multidim(n_dims=ndmin, **kwargs)
        multi=['multidim: '+str(ndmin)]
        WA_list=[WA_mul]
    
    multi.extend(methods)
    methods=multi.copy()

    WA_mat=assign_wells_mat(**kwargs)

    # random assignment

    WA_ran=assign_wells_random(**kwargs)

    # STD asignment 
    WA_std=assign_wells_STD(**kwargs)

    

    # chinese trick assignment
    WA_chin=assign_wells_chinese(**kwargs)


    WA_list.extend([WA_mat, WA_ran,WA_std, WA_chin])

    if kwargs['differentiate']<2:
        WA_bin=assign_wells_L(**kwargs)
        methods.append('Binary')
        WA_list.append(WA_bin)

    #hierarchical
        
    Hier=calculate_metrics_hierarchical(**kwargs)
    #return([BM[0], BM[1],layers, MC, details ])

    ls_met=[]
    ls_names_met=['mean_experiments', 'max_compounds_per_well', 'n_wells', 'percentage_check', 'mean_extra_exp', 'mean_steps']
    for method, WA in zip(methods, WA_list):
        mean_exp, extra_exp,  _, perc_check= mean_metrics(WA, **kwargs)
        n_wells=WA.shape[1]
        M_exp=np.round(mean_exp, 2)
        max_comp=np.max(np.sum(WA, axis=0))
        ls_met.append([M_exp, max_comp, n_wells, int(perc_check),  extra_exp,1+perc_check/100])
    ls_met.append(Hier[:-1])
    full_methods=methods.copy()
    full_methods.append('Hierarchical')
    df_met=pd.DataFrame(ls_met)

    dict_wa={method: WA for method, WA in zip(methods, WA_list)}
    dict_wa.update({'Hierarchical':Hier[5]})

    idx_renamer={i:j for i,j in zip(df_met.index, full_methods)}
    col_renamer={i:j for i,j in zip(df_met.columns, ls_names_met)}
    df_met.rename(index=idx_renamer, columns=col_renamer, inplace=True)

    ret_wa= kwargs['return_wa'] 
    if ret_wa:
        return df_met, dict_wa
    return df_met

In [20]:
WA=assign_wells_mat(n_compounds=296)

In [21]:
WA.shape

(296, 35)

In [22]:
scrambler[2]

array([[  0,   1],
       [  0,   2],
       [  0,   3],
       ...,
       [292, 295],
       [293, 295],
       [294, 295]])

In [25]:
np.bool(np.sum(WA[scrambler[3]], axis=1))

array([[ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [26]:
is_consistent_precomp(WA,3,scrambler)

(False,
 array([[ True, False, False, ..., False, False, False],
        [ True, False, False, ..., False, False, False],
        [ True, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 array([1, 1, 1, ..., 6, 6, 6]))

In [ ]:
def mean_metrics_precomp(well_assigner, differentiate, scrambler, **kwargs):
    BT=well_assigner.shape[1]
    _,_, counts= is_consistent_precomp(well_assigner, differentiate, scrambler) 
    ET=extra_tests(counts)   
    rounds=np.sum(counts>1)/np.sum(counts>0)+1
    p_check=np.round(np.sum(counts[counts>1])/np.sum(counts)*100)
    return BT+ET, ET,  rounds, p_check

def decode_precomp(well_assigner:np.ndarray, readout:np.ndarray, differentiate:int, scrambler:dict) -> list:
    N=well_assigner.shape[0]
    for i in range(differentiate):
        resulti=[]
        diff=i+1
        if diff ==1:
            resulti.extend(list(range(well_assigner.shape[0])))
            full_well_assigner=well_assigner.copy()
        else:
            this_sc=scrambler[diff]
            full_well_assigner=np.concatenate((full_well_assigner,np.bool(np.sum(well_assigner[this_sc], axis=1))))
    idxs=[i for i in range(full_well_assigner.shape[0]) if np.array_equal(full_well_assigner[i,:],readout)]
    if len(idxs)==0:
        print('No match')
        return(-1)
    
    return [resulti[i] for i in idxs]

In [28]:
def decode_precomp(well_assigner:np.ndarray, readout:np.ndarray, differentiate:int, scrambler:dict) -> list:
    N=well_assigner.shape[0]
    for i in range(differentiate):
        resulti=[]
        diff=i+1
        if diff ==1:
            resulti.extend(list(range(well_assigner.shape[0])))
            full_well_assigner=well_assigner.copy()
        else:
            this_sc=scrambler[diff]
            full_well_assigner=np.concatenate((full_well_assigner,np.bool(np.sum(well_assigner[this_sc], axis=1))))
    idxs=[i for i in range(full_well_assigner.shape[0]) if np.array_equal(full_well_assigner[i,:],readout)]
    if len(idxs)==0:
        print('No match')
        return(-1)
    
    return [resulti[i] for i in idxs]

In [31]:
mean_metrics_precomp(WA,3,scrambler)

(np.float64(39.79847019272499),
 np.float64(4.798470192724995),
 np.float64(1.956555662663945),
 np.float64(99.0))

In [ ]:
def full_deterministic_comparison(**kwargs):
    methods=['matrix', 'random', 'STD', 'Chinese trick']
    # matrix assignment
    

    # multidimensional matrix
    if 'n_dims' in kwargs.keys():
        WA_mul=assign_wells_multidim(**kwargs)
        ndmin=kwargs['n_dims']
        multi=['multidim: '+str(ndmin)]
        WA_list=[WA_mul]
    elif 'all_dims' in kwargs.keys():
        if kwargs['all_dims']:
            WA_list=[]
            multi=[]
            for i in np.arange(2,int(np.ceil(np.log(kwargs['n_compounds'])/np.log(2)))):
                if i>kwargs['max_dims']:
                    continue
                WA_mul=assign_wells_multidim(n_dims=i, **kwargs)
                WA_list.append(WA_mul)
                multi.append('multidim: '+str(i))

        else:
            ndmin= find_dims(**kwargs)
            WA_mul=assign_wells_multidim(n_dims=ndmin, **kwargs)
            multi=['multidim: '+str(ndmin)]
            WA_list=[WA_mul]
            


    else:
        ndmin= find_dims(**kwargs)
        WA_mul=assign_wells_multidim(n_dims=ndmin, **kwargs)
        multi=['multidim: '+str(ndmin)]
        WA_list=[WA_mul]
    
    multi.extend(methods)
    methods=multi.copy()

    WA_mat=assign_wells_mat(**kwargs)

    # random assignment

    WA_ran=assign_wells_random(**kwargs)

    # STD asignment 
    WA_std=assign_wells_STD(**kwargs)

    

    # chinese trick assignment
    WA_chin=assign_wells_chinese(**kwargs)


    WA_list.extend([WA_mat, WA_ran,WA_std, WA_chin])

    if kwargs['differentiate']<2:
        WA_bin=assign_wells_L(**kwargs)
        methods.append('Binary')
        WA_list.append(WA_bin)

    #hierarchical
        
    Hier=calculate_metrics_hierarchical(**kwargs)
    #return([BM[0], BM[1],layers, MC, details ])

    ls_met=[]
    ls_names_met=['mean_experiments', 'max_compounds_per_well', 'n_wells', 'percentage_check', 'mean_extra_exp', 'mean_steps']
    for method, WA in zip(methods, WA_list):
        mean_exp, extra_exp,  _, perc_check= mean_metrics(WA, **kwargs)
        n_wells=WA.shape[1]
        M_exp=np.round(mean_exp, 2)
        max_comp=np.max(np.sum(WA, axis=0))
        ls_met.append([M_exp, max_comp, n_wells, int(perc_check),  extra_exp,1+perc_check/100])
    ls_met.append(Hier[:-1])
    full_methods=methods.copy()
    full_methods.append('Hierarchical')
    df_met=pd.DataFrame(ls_met)

    dict_wa={method: WA for method, WA in zip(methods, WA_list)}
    dict_wa.update({'Hierarchical':Hier[5]})

    idx_renamer={i:j for i,j in zip(df_met.index, full_methods)}
    col_renamer={i:j for i,j in zip(df_met.columns, ls_names_met)}
    df_met.rename(index=idx_renamer, columns=col_renamer, inplace=True)

    ret_wa= kwargs['return_wa'] 
    if ret_wa:
        return df_met, dict_wa
    return df_met

In [ ]:
with open('./combinations/N_294.pk', 'rb') as handle:
    f1=pickle.load(handle)

In [ ]:
F2=iterative_add_N(f1,400, save=False)

In [ ]:
F1=add_N(F_dict,6,200)

In [ ]:
with open('./cmb_dict.pk', 'wb') as handle:
    pickle.dump(F1, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
len(F1)

In [ ]:
F1[1000][4].shape

In [ ]:
len(list(itertools.combinations(np.arange(1000),4)))

In [ ]:
tri=add_1(dct_cmbn)

In [ ]:
tri

In [ ]:
np.array(list(itertools.combinations(np.arange(6),2))).shape

In [ ]:
pk_dir='./diff_1/20-47_step5.pk'

In [ ]:
pk_dir='./single_method/multidim_3/multidim_3__20-47_step5.pk'

In [ ]:
with open(pk_dir, 'rb') as handle:
    f1=pickle.load(handle)

In [ ]:
f1[45]['WA'].shape

In [ ]:
# To merge .pk files

ls_names_met=['mean_experiments', 'max_compounds_per_well', 'n_wells', 'percentage_check', 'mean_extra_exp']
full_dict={}
for j in range(1,5):
    base_dir='diff_'+str(j)
    diff_dict={}
    filenames = next(os.walk(base_dir), (None, None, []))[2]
    for file in filenames:
        full_dir=os.path.join(base_dir,file)
        with open(full_dir, 'rb') as handle:
            f1=pickle.load(handle)
        for i in list(f1):
            if i=='kwargs':
                del f1['kwargs']
                continue
            else:
                WA=assign_wells_mat(i)
                f1[i][1].update({'matrix':WA})
                mean_exp, extra_exp,  _, perc_check= mean_metrics(WA, differentiate=j)
                n_wells=WA.shape[1]
                M_exp=np.round(mean_exp, 2)
                max_comp=np.max(np.sum(WA, axis=0))
                ls_met=[M_exp, max_comp, n_wells, perc_check,  extra_exp]
                f2=f1[i][0].drop(labels='matrix')
                idxs=list(f2.index)
                idxs+=['matrix']
                r_idxs=idxs[-1:]+idxs[:-1]
                dict_met={nm:list(f2[nm])+[val] for nm,val in zip(ls_names_met, ls_met)}

                f1[i][0]=pd.DataFrame(data=dict_met, index=idxs).reindex(r_idxs)

                
                f1[i][0]=f1[i][0].round({'mean_extra_exp':2})
            
        diff_dict.update(f1)
    full_str='Differentiate '+str(j)
    full_dict.update({full_str:diff_dict})

    
    


In [ ]:
#output as pickle file
full_dir='Final_precomputed_file.pk'

#with open(full_dir, 'wb') as handle:
    #pickle.dump(full_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# To merge files generated with inline_print
all_methods = ['matrix', 'multidim', 'random', 'STD', 'CT', 'Binary']
results = {}

for method_dir in all_methods:
    filenames = next(os.walk(method_dir), (None, None, []))[2]
    metrics_list = []
    for file in filenames:
        if file.endswith(".txt"):
            try:
                metrics = extract_metrics(file)
                metrics["method"] = method_dir
                metrics_list.append(metrics)
            except ValueError as e:
                print(e)
    
    if metrics_list:
        df = pd.DataFrame(metrics_list)
        df = df.sort_values(by="NS")
        
        # Group by diff value
        for diff_value, group_df in df.groupby("diff"):
            group_df = group_df.reset_index(drop=True)
            
            if diff_value not in results:
                results['Differentiate ' + str(diff_value)] = {}
                
            for ns_value, ns_group_df in group_df.groupby("NS"):
                ns_group_df = ns_group_df.drop(columns=["diff", "NS"]) 
                ns_group_df = ns_group_df.rename(columns={"NW": "n_wells", "MS": "max_compounds_per_well"})
               
                ns_group_df = ns_group_df.set_index(["method"])
                
                if ns_value not in results['Differentiate ' + str(diff_value)]:
                    results['Differentiate ' + str(diff_value)][ns_value] = [ns_group_df, {}]
                else:
                    existing_df, empty_dict = results['Differentiate ' + str(diff_value)][ns_value]
                    
                    merged_df = pd.merge(existing_df.reset_index(), ns_group_df.reset_index(),
                                         on=["method", "n_wells", "max_compounds_per_well"], how="outer", indicator=True)
                    
                    # Filter out rows that are duplicates (the '_merge' column indicates if it's from both dataframes)
                    new_entries_df = merged_df[merged_df['_merge'] == 'right_only'].drop(columns=["_merge"]).set_index("method")
                    existing_entries_df = merged_df[merged_df['_merge'] == 'both'].drop(columns=["_merge"]).set_index("method")
                    
                    # Concatenate only new entries
                    combined_df = pd.concat([existing_entries_df, new_entries_df]).drop_duplicates().reset_index(drop=True)
                    
                    # Reassign the combined dataframe
                    results['Differentiate ' + str(diff_value)][ns_value] = [combined_df, {}]
            

In [ ]:

# Output as pickle file
full_dir = 'Final_inline_precomputed_file.pk'

# First check if file already exists
# Merge new and old if it does
if os.path.exists(full_dir):
    with open(full_dir, "rb") as f:
        existing_results = pickle.load(f)
        
    for diff_value, ns_values in results.items():
        if diff_value not in existing_results:
            existing_df, existing_empty_dict = existing_results[diff_value][ns_value]
            
            for ns_value, (new_df, empty_dict) in ns_values.items():
                if ns_value in existing_results[diff_value]:
                    existing_df, existing_empty_dict = existing_results[diff_value][ns_value]
                
            
                    merged_df = pd.merge(existing_df.reset_index(), new_df.reset_index(),
                                         on=["method", "n_wells", "max_compounds_per_well"], how="outer", indicator=True)
                    
                    new_entries_df = merged_df[merged_df['_merge'] == 'right_only'].drop(columns=["_merge"]).set_index("method")
                    existing_entries_df = merged_df[merged_df['_merge'] == 'both'].drop(columns=["_merge"]).set_index("method")
                    
                    combined_df = pd.concat([existing_entries_df, new_entries_df]).drop_duplicates().reset_index(drop=True)
                    existing_results[diff_value][ns_value] = [combined_df.set_index(["method"]), {}]
                else:
                    existing_results[diff_value][ns_value] = [new_df, empty_dict]
            
                
    merged_results = existing_results
    
else:
    merged_results = results

#with open(full_dir, 'wb') as handle:
    #pickle.dump(merged_results, handle, protocol=pickle.HIGHEST_PROTOCOL)